# Bayesian Analysis of Altruistic Food Sharing in Rats

This notebook replicates the statistical analyses from the publication "Failure to find altruistic food sharing in rats" (*Frontiers in Psychology*, 2021). The goal is to translate the original R/Stan analysis into a Python-based workflow using Pandas and CmdStanPy.

**Models presented:**

1. A **hierarchical binomial logistic regression** to model the choice between food and social interaction.
2. A **hierarchical negative binomial regression** to model the rate of food and social responses.
3. A **hierarchical negative binomial regression** to model food pellet consumption and sharing.

In [ ]:
import pandas as pd
from cmdstanpy import CmdStanModel
import os
import logging
import pandas as pd

# Set the display format for floating-point numbers to 3 decimal places
pd.options.display.float_format = '{:.3f}'.format

# Get the logger for CmdStanPy
logger = logging.getLogger('cmdstanpy')

# Set the logging level to ERROR to hide INFO and WARNING messages
logger.setLevel(logging.ERROR)

print("--- 1. Loading and Preparing Data ---")

# Load the dataset
behav_data = pd.read_csv("final versions/Data and analysis code/Raw_data.csv")

# Create a 1-based integer subject ID for Stan
behav_data['Subject'] = behav_data['Rat'].astype('category').cat.codes + 1

--- 1. Loading and Preparing Data ---
Data loaded successfully.


,Rat,Condition,FoodAmmt,Social,Food,Sharing,TimesShared,PelletsLeft,Subject
0,4,1,1,5,78,0,-1,-1,1
1,4,1,1,0,127,0,-1,-1,1
2,4,1,1,10,69,0,-1,-1,1
3,4,1,1,7,78,0,-1,-1,1
4,4,1,1,22,38,0,-1,-1,1


## 2. Model 1: Food vs. Social Choice

This model uses a hierarchical binomial logistic regression to estimate the rate of food choice. As noted in the paper, Condition 5 is excluded because preference is undefined when one of the response options is unavailable.

In [2]:
# Cell 4: Model 1 - Code

print("--- 2. Running Model 1: Food vs. Social Choice (Binomial) ---")

# Stan code with the for-loop in the model block
pair_model_code = """
data{
	int<lower=1> N;         // Number of observations
	int<lower=1> S;         // Number of subjects
	array[N] int<lower=1> subID;  // Subject IDs
	int<lower=1> C;         // Number of conditions
	array[N] int<lower=1> condID; // Condition ID
	array[N] int<lower=0> choice; // Number of choices per session
	array[N] int<lower=0> food;   // Number of food choices (such that choose[s]-food[s]=social[s])
}
//
transformed data {
	vector[S] u;
	for (s in 1:S) {
		u[s] = 1;
	}
}
//
parameters{
	matrix[C, S] z;                   // beta proxy
	cholesky_factor_corr[C] L_Omega;  // prior correlation
	vector<lower=0>[C] tau;           // prior scale
	row_vector[C] gamma;              // population means
}
//
transformed parameters{
	matrix[S,C] beta;                 // individual coefficients for each subject in each condition
	beta = u * gamma + (diag_pre_multiply(tau,L_Omega) * z)';
}
//
model{
	array[N] real mu;

	to_vector(z) ~ normal(0,1);
	L_Omega ~ lkj_corr_cholesky(2);
	tau ~ exponential(1.5);
	to_vector(gamma) ~ normal(0,1.5);

	for ( n in 1:N ) {
		mu[n] = beta[subID[n],condID[n]];
	}
	food ~ binomial_logit(choice,mu);
}
"""

# Write model to a .stan file
stan_file_pair = "pair_multilevel_model.stan"
with open(stan_file_pair, "w") as f:
    f.write(pair_model_code)

# Compile the Stan model
pair_model = CmdStanModel(stan_file=stan_file_pair)

# Prepare data: filter out condition 5
data_subset = behav_data[behav_data['Condition'] != 5].copy()
data_subset['Condition_new'] = data_subset['Condition']
data_subset.loc[data_subset['Condition'] > 5, 'Condition_new'] = data_subset['Condition'] - 1

stan_data_choice = {
    "N": len(data_subset),
    "S": data_subset['Subject'].nunique(),
    "subID": data_subset['Subject'].tolist(),
    "C": data_subset['Condition_new'].nunique(),
    "condID": data_subset['Condition_new'].tolist(),
    "choice": (data_subset['Social'] + data_subset['Food']).tolist(),
    "food": data_subset['Food'].tolist()
}

# Sample from the posterior distribution
choice_fit = pair_model.sample(
    data=stan_data_choice, chains=4, parallel_chains=4,
    iter_warmup=1500, iter_sampling=1500,
    adapt_delta=0.99, max_treedepth=15, 
	show_progress = False, refresh = 3000
)
    
print("Model 1 Sampling Complete.")
# Display summary for the 'gamma' parameter
fitsum = choice_fit.summary()
print(fitsum[fitsum.index.str.startswith("gamma")])

21:33:08 - cmdstanpy - INFO - compiling stan file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/pair_multilevel_model.stan to exe file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/pair_multilevel_model


--- 2. Running Model 1: Food vs. Social Choice (Binomial) ---


21:33:20 - cmdstanpy - INFO - compiled model executable: /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/pair_multilevel_model
21:33:21 - cmdstanpy - INFO - CmdStan start processing
21:33:21 - cmdstanpy - INFO - Chain [1] start processing
21:33:21 - cmdstanpy - INFO - Chain [2] start processing
21:33:21 - cmdstanpy - INFO - Chain [3] start processing
21:33:21 - cmdstanpy - INFO - Chain [4] start processing
21:33:32 - cmdstanpy - INFO - Chain [2] done processing
21:33:33 - cmdstanpy - INFO - Chain [4] done processing
21:33:33 - cmdstanpy - INFO - Chain [1] done processing
21:33:35 - cmdstanpy - INFO - Chain [3] done processing
21:33:35 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'pair_multilevel_model.stan', line 35, column 1 to column 32)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'pair_multileve

Model 1 Sampling Complete.
           Mean  MCSE  StdDev   MAD     5%    50%   95%  ESS_bulk  ESS_tail  \
gamma[1]  2.281 0.011   0.586 0.489  1.220  2.361 3.081  2934.730  3532.210   
gamma[2]  1.735 0.012   0.659 0.564  0.548  1.813 2.668  3388.860  3309.530   
gamma[3]  0.657 0.008   0.415 0.326 -0.020  0.669 1.279  3102.210  2734.010   
gamma[4] -0.051 0.006   0.265 0.179 -0.465 -0.046 0.341  2971.950  2187.250   
gamma[5]  0.095 0.003   0.188 0.140 -0.171  0.097 0.362  5231.790  3882.760   
gamma[6]  0.259 0.007   0.430 0.358 -0.439  0.267 0.933  3641.560  3457.170   

          ESS_bulk/s  R_hat  
gamma[1]     115.051  1.002  
gamma[2]     132.855  1.000  
gamma[3]     121.617  1.001  
gamma[4]     116.511  1.000  
gamma[5]     205.104  1.001  
gamma[6]     142.761  1.001  


## 3. Model 2: Response Rates by Type

This model estimates the absolute number of food and social choices per session using a hierarchical negative binomial regression to account for overdispersion in the count data.

In [3]:
print("--- 3. Running Model 2: Response Rates (Negative Binomial) ---")

resp_rate_model_code = """
data {
    int<lower=1> N;
    int<lower=1> S;
    array[N] int<lower=1> subID;
    int<lower=1> C;
    array[N] int<lower=1> condID;
    array[N] int<lower=0> FC;
    array[N] int SC;
}
parameters {
    matrix[2*C, S] z;
    cholesky_factor_corr[2*C] L_Omega;
    vector<lower=0>[2*C] tau;
    row_vector[2*C] gamma;
    real<lower=0> overdisp;
}
transformed parameters {
    matrix[S, 2*C] beta;
    beta = rep_matrix(gamma, S) + (diag_pre_multiply(tau, L_Omega) * z)';
}
model {
    to_vector(z) ~ std_normal();
    L_Omega ~ lkj_corr_cholesky(2);
    tau ~ exponential(1.5);
    to_vector(gamma) ~ normal(1, 1.5);
    overdisp ~ exponential(1);
    
    for (n in 1:N) {
      FC[n] ~ neg_binomial_2_log(beta[subID[n], condID[n]], overdisp);
      if (SC[n] >= 0) {
        SC[n] ~ neg_binomial_2_log(beta[subID[n], condID[n] + C], overdisp);
      }
    }
}
"""
stan_file_resp_rate = "resp_rate_multilevel_model.stan"
with open(stan_file_resp_rate, "w") as f:
    f.write(resp_rate_model_code)
        
resp_rate_model = CmdStanModel(stan_file=stan_file_resp_rate)

stan_data_resp_rate = {
    "N": len(behav_data), "S": behav_data['Subject'].nunique(),
    "subID": behav_data['Subject'].tolist(), "C": behav_data['Condition'].nunique(),
    "condID": behav_data['Condition'].tolist(), "FC": behav_data['Food'].tolist(),
    "SC": behav_data['Social'].tolist()
}

resp_rate_fit = resp_rate_model.sample(
    data=stan_data_resp_rate, chains=4, parallel_chains=4,
    iter_warmup=1500, iter_sampling=1500,
    adapt_delta=0.99, max_treedepth=15, 
	show_progress = False, refresh = 3000
)
    
print("Model 2 Sampling Complete.")
fitsum = resp_rate_fit.summary()
print(fitsum[fitsum.index.str.startswith("gamma")])

21:33:35 - cmdstanpy - INFO - compiling stan file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/resp_rate_multilevel_model.stan to exe file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/resp_rate_multilevel_model


--- 3. Running Model 2: Response Rates (Negative Binomial) ---


21:33:46 - cmdstanpy - INFO - compiled model executable: /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/resp_rate_multilevel_model
21:33:47 - cmdstanpy - INFO - CmdStan start processing
21:33:47 - cmdstanpy - INFO - Chain [1] start processing
21:33:47 - cmdstanpy - INFO - Chain [2] start processing
21:33:47 - cmdstanpy - INFO - Chain [3] start processing
21:33:47 - cmdstanpy - INFO - Chain [4] start processing
21:34:54 - cmdstanpy - INFO - Chain [1] done processing
21:34:59 - cmdstanpy - INFO - Chain [4] done processing
21:35:17 - cmdstanpy - INFO - Chain [2] done processing
21:35:21 - cmdstanpy - INFO - Chain [3] done processing
21:35:21 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'resp_rate_multilevel_model.stan', line 24, column 4 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'resp

Model 2 Sampling Complete.
           Mean  MCSE  StdDev   MAD     5%   50%   95%  ESS_bulk  ESS_tail  \
gamma[1]  4.621 0.011   0.491 0.283  3.707 4.721 5.161  2407.840  2042.430   
gamma[2]  4.219 0.009   0.450 0.275  3.362 4.308 4.734  2749.760  2879.620   
gamma[3]  3.269 0.005   0.261 0.153  2.827 3.308 3.584  3149.620  2622.050   
gamma[4]  2.468 0.007   0.361 0.261  1.858 2.502 2.952  2928.160  2584.440   
gamma[5]  2.819 0.004   0.197 0.137  2.509 2.828 3.086  3644.290  2868.680   
gamma[6]  2.442 0.004   0.219 0.153  2.090 2.460 2.728  4226.740  2795.620   
gamma[7]  2.580 0.004   0.200 0.130  2.287 2.594 2.828  3504.610  2353.480   
gamma[8]  2.098 0.008   0.414 0.314  1.387 2.137 2.672  3304.460  3118.000   
gamma[9]  2.116 0.008   0.522 0.453  1.214 2.153 2.881  4378.640  3854.100   
gamma[10] 2.410 0.008   0.463 0.375  1.587 2.451 3.060  3532.590  3235.530   
gamma[11] 2.408 0.008   0.461 0.385  1.591 2.450 3.076  3839.970  3413.480   
gamma[12] 0.992 0.012   1.478 1.454 -

## 4. Model 3: Food Intake and Sharing

The final model analyzes the counts of food pellets that were consumed, "shared," or left behind. "Sharing" was defined as the focal rat producing food and then releasing its partner while food remained available.

In [4]:
print("--- 4. Running Model 3: Food Intake (Negative Binomial) ---")
    
intake_model_code = """
data {
    int<lower=1> N;
    int<lower=1> S;
    array[N] int<lower=1> subID;
    int<lower=1> C;
    array[N] int<lower=1> condID;
    array[N] int<lower=0> CP;
    array[N] int LP;
    array[N] int SP;
}
parameters {
    matrix[3*C, S] z;
    cholesky_factor_corr[3*C] L_Omega;
    vector<lower=0>[3*C] tau;
    row_vector[3*C] gamma;
    real<lower=0> overdisp;
}
transformed parameters {
    matrix[S, 3*C] beta;
    beta = rep_matrix(gamma, S) + (diag_pre_multiply(tau, L_Omega) * z)';
}
model {
    to_vector(z) ~ std_normal();
    L_Omega ~ lkj_corr_cholesky(2);
    tau ~ exponential(1.5);
    to_vector(gamma) ~ normal(0, 1.5);
    overdisp ~ exponential(1);
    
    for (n in 1:N) {
      CP[n] ~ neg_binomial_2_log(beta[subID[n], condID[n]], overdisp);
      if (LP[n] >= 0) {
        LP[n] ~ neg_binomial_2_log(beta[subID[n], condID[n] + C], overdisp);
      }
      if (SP[n] >= 0) {
        SP[n] ~ neg_binomial_2_log(beta[subID[n], condID[n] + (2*C)], overdisp);
      }
    }
}
"""
stan_file_intake = "intake_multilevel_model.stan"
with open(stan_file_intake, "w") as f:
    f.write(intake_model_code)
        
intake_model = CmdStanModel(stan_file=stan_file_intake)

stan_data_intake = {
    "N": len(behav_data), "S": behav_data['Subject'].nunique(),
    "subID": behav_data['Subject'].tolist(), "C": behav_data['Condition'].nunique(),
    "condID": behav_data['Condition'].tolist(),
    "CP": (behav_data['Food'] * behav_data['FoodAmmt']).tolist(),
    "SP": behav_data['Sharing'].tolist(),
    "LP": behav_data['PelletsLeft'].tolist()
}
    
intake_fit = intake_model.sample(
    data=stan_data_intake, chains=4, parallel_chains=4,
    iter_warmup=1500, iter_sampling=1500,
    adapt_delta=0.99, max_treedepth=15, 
	show_progress = False, refresh = 3000
)
    
print("Model 3 Sampling Complete.")
fitsum = intake_fit.summary()
print(fitsum[fitsum.index.str.startswith("gamma")])

21:35:23 - cmdstanpy - INFO - compiling stan file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/intake_multilevel_model.stan to exe file /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/intake_multilevel_model


--- 4. Running Model 3: Food Intake (Negative Binomial) ---


21:35:34 - cmdstanpy - INFO - compiled model executable: /Users/Matt/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/Research/2017/RatSharing/intake_multilevel_model
21:35:34 - cmdstanpy - INFO - CmdStan start processing
21:35:34 - cmdstanpy - INFO - Chain [1] start processing
21:35:34 - cmdstanpy - INFO - Chain [2] start processing
21:35:34 - cmdstanpy - INFO - Chain [3] start processing
21:35:34 - cmdstanpy - INFO - Chain [4] start processing
21:37:01 - cmdstanpy - INFO - Chain [3] done processing
21:37:02 - cmdstanpy - INFO - Chain [4] done processing
21:37:03 - cmdstanpy - INFO - Chain [2] done processing
21:37:09 - cmdstanpy - INFO - Chain [1] done processing
21:37:09 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'intake_multilevel_model.stan', line 25, column 4 to column 35)
	Exception: lkj_corr_cholesky_lpdf: Random variable[2] is 0, but must be positive! (in 'intake_mul

Model 3 Sampling Complete.
            Mean  MCSE  StdDev   MAD     5%    50%    95%  ESS_bulk  ESS_tail  \
gamma[1]   4.435 0.022   0.726 0.318  2.909  4.653  5.060  1812.570  1272.410   
gamma[2]   4.701 0.020   0.748 0.320  3.111  4.942  5.329  1807.640  1504.420   
gamma[3]   4.562 0.013   0.444 0.173  3.759  4.668  4.942  2103.200  1257.550   
gamma[4]   3.708 0.015   0.521 0.295  2.740  3.825  4.258  1968.950  1485.740   
gamma[5]   4.350 0.013   0.358 0.164  3.891  4.408  4.676  2015.650  1028.950   
gamma[6]   3.962 0.011   0.378 0.173  3.352  4.037  4.306  2221.310  1159.590   
gamma[7]   4.146 0.005   0.241 0.147  3.740  4.179  4.422  3693.350  2126.130   
gamma[8]   0.018 0.013   1.459 1.464 -2.423  0.035  2.394 12185.200  3972.110   
gamma[9]   0.016 0.014   1.538 1.487 -2.556  0.027  2.541 13129.400  4159.120   
gamma[10] -0.007 0.014   1.484 1.510 -2.461 -0.012  2.464 11615.300  4225.090   
gamma[11] -0.009 0.015   1.513 1.474 -2.523 -0.036  2.511 10792.400  4450.950   
g